***
# **Screener**

Operation:
1. Define function ```buysig()``` which scrapes stock data, calculates buy signals and returns pandas series holding information when the last buy signal happend and distance from today
2. Filter results to show only buy signals from last two days
3. Saves results into excel sheet

In [1]:
# imports
from scraper import stock_daily
from IPython.display import display
import datetime as dt
import time
import numpy as np
import pandas as pd
import indicators as ind

***
## **Define ```buysig()```**

* Input: empty pandas series with predefined columns
* Output: said series with filled results

Scrapes data, calculate indicators and buy signals (depending on strategy, supports multiple strategies), saves results into series and outputs it. Currently supports following strategies:

* SVF: Buy signal dependent on fast (5 days) and slow (21 days) stochastic oscillator and their crosses. Censored by trends in the VFI (uptrending histogram or VFI above 0)
* S&F: Buy signals generated based on increasing momentum (calculated as a difference between EMA3-EMA10 and EMA3-EMA26). Censored by trends in the VFI
* SMA bounces: Outputs stocks which bounced from SMA200 / SMA100 / SMA50 during last trading day. Stocks are required to be in general uptrend and bounced already.

In [2]:
def buysig(result):
    # SCRAPING
    try:
        stock = stock_daily(result["ticker"], save=False)
    except:
        print(result["ticker"] + ": Exception occured during data scraping, skipped.")
        return result

    # CREATING BUY SIGNALS
    # Strategy SVF
    # calculating indicators
    vfi = ind.vfi(stock.data, period=30, coef=0.2, vcoef=1.5)
    fs = ind.stoch(stock.data, period=5, sk=2, sd=3)
    ss = ind.stoch(stock.data, period=21, sk=2, sd=5)
    # calculating VFI histogram trend
    window = 2
    vfi_hist = vfi["histogram"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    with np.errstate(invalid="ignore"):
        vfi_hist = vfi_hist > 0
        # VFI trend
        vfi_trend = vfi["vfi"] > vfi["vfi_smooth"]
    vfi_conf = np.logical_or(vfi_hist, vfi_trend)
    # calculating fast stochastic trend
    window = 4
    fs_conf = fs["k"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    with np.errstate(invalid="ignore"):
        fs_conf = fs_conf > 0
    # buy signals
    conditions = np.logical_and((ss["k"] > ss["d"]).to_numpy(), (ss["d"] >= 0).to_numpy())
    bss = np.concatenate((np.array([0]), (conditions[:-1] < conditions[1:]))).astype("int")
    # finalize buy signals
    svf = np.logical_and(np.logical_and(fs_conf, vfi_conf), bss)
    # -------
    # Strategy S&F Oscillator
    ema3 = ind.ema(stock.data, 3, price="Typical")["EMA"].to_numpy()
    ema10 = ind.ema(stock.data, 10, price="Open")["EMA"].to_numpy()
    ema26 = ind.ema(stock.data, 26, price="Close")["EMA"].to_numpy()
    # buy signal
    bcurve = ema3-ema10
    ccurve = ema3-ema26
    lim = 0.1*stock.data["Close"].to_numpy()
    with np.errstate(invalid='ignore'):
        bmask = bcurve > 0
        cmask = (ccurve > -lim).astype("int")
    bsig = np.concatenate((np.array([0]), (bmask[:-1] < bmask[1:]))).astype("int")
    bsig = np.logical_and(bsig, cmask).astype("int")
    # VFI mask
    vfimask = (vfi["vfi"] > 0).to_numpy()
    vfimask = np.logical_or(vfimask, (vfi["histogram"] > 0).to_numpy())
    saf = np.logical_and(bsig, vfimask).astype("int")

    # ------
    # Strategy SMA bounce
    # SMA200
    sma200b = 0
    try:
        smaline = ind.sma(stock.data, 200, price="Close")["SMA"]
        bounces = ind.sma_bounces(stock.data, smaline)
        # we look at stock that are above the given SMA line for the past 100 days
        dropsBelowSMA = np.where(stock.data["Close"][-200:] - smaline[-200:] <0)[0]
        # previous bounces from SMA, above SMA for 100 days and just above SMA the previous day
        if bounces.size > 1 and dropsBelowSMA.size == 0 and (stock.data["Close"].iat[-1]-smaline.iat[-1])/smaline.iloc[-1]<0.05: 
            sma200b = 1
    except:
        print(result["ticker"] + ": problem during SMA200 bounce calculation")
    result["SMA200_bounce"] = sma200b
    # SMA100
    sma100b = 0
    try:
        smaline = ind.sma(stock.data, 100, price="Close")["SMA"]
        bounces = ind.sma_bounces(stock.data, smaline)
        # we look at stock that are above the given SMA line for the past 100 days
        dropsBelowSMA = np.where(stock.data["Close"][-200:] - smaline[-200:] <0)[0]
        # previous bounces from SMA, above SMA for 100 days and just above SMA the previous day
        if bounces.size > 1 and dropsBelowSMA.size == 0 and (stock.data["Close"].iat[-1]-smaline.iat[-1])/smaline.iloc[-1]<0.05: 
            sma100b = 1
    except:
        print(result["ticker"] + ": problem during SMA100 bounce calculation")
    result["SMA100_bounce"] = sma100b
    # SMA50
    sma50b = 0
    try:
        smaline = ind.sma(stock.data, 50, price="Close")["SMA"]
        bounces = ind.sma_bounces(stock.data, smaline)
        # we look at stock that are above the given SMA line for the past 100 days
        dropsBelowSMA = np.where(stock.data["Close"][-200:] - smaline[-200:] < 0)[0]
        # previous bounces from SMA, above SMA for 100 days and just above SMA the previous day
        if bounces.size > 1 and dropsBelowSMA.size == 0 and (stock.data["Close"].iat[-1]-smaline.iat[-1])/smaline.iloc[-1]<0.05: 
            sma50b = 1
    except:
        print(result["ticker"] + ": problem during SMA50 bounce calculation")
    result["SMA50_bounce"] = sma50b
    # SAVING DATA
    # date of last buy signal and distance from today
    # SVF
    try:
        lastindex = np.squeeze(np.where(svf == True))[-1]
        result["buy_svf"] = stock.data.loc[lastindex,"Date"]
        result["dist_svf"] = stock.data.index[-1] - lastindex
    except:
        print(result["ticker"] + ": No SVF signals generated")
    # S&F
    try:
        lastindex = np.squeeze(np.where(saf == True))[-1]
        result["buy_saf"] = stock.data.loc[lastindex,"Date"]
        result["dist_saf"] = stock.data.index[-1] - lastindex
    except:
        print(result["ticker"] + ": No S&F signals generated")

    return result

***
## **Run the analysis**

Tickers are stored in ```tickers.xlsx``` excel file where multiple sheets are defined. By modifying the *sheets* variable, one can choose which stocks to scan through

Sheets in the excel:
1. spy - holds all stock tickers present in the SPY ETF
2. iwm - holds top 600 holdings in the IWM ETF which follows Russell 2000
3. watchlist - list of stocks I'm personally interested in and found somewhere else, on fintwit, reddit, my own research
4. longs - stocks I have high conviction in and want to hold long term


In [3]:
# which sheets to use:
# sheets = ["spy", "iwm", "watchlist", "longs", "midcap", "largecap"] # specific scan
sheets = ["filtered"] # full nasdaq, filtered by volume and volatility
# sheets = ["test"]

# read tickers from excel file
excel = pd.read_excel("tickers.xlsx", sheet_name=sheets)

In [4]:
# define empty dataframe
data = pd.DataFrame(columns=["ticker", "type", "buy_svf", "dist_svf", "buy_saf", "dist_saf", "SMA50_bounce", "SMA100_bounce", "SMA200_bounce"])
# run analysis
for i in sheets:
    for j in excel[i]["ticker"].to_list():
        # define series
        ser = {"ticker": j, "type": i, "buy_svf": np.nan, "dist_svf": np.nan, "buy_saf": np.nan, "dist_saf": np.nan, "SMA50_bounce": np.nan, "SMA100_bounce": np.nan, "SMA200_bounce": np.nan}
        ser = pd.Series(data=ser)
        # delay for better scrapping (without this, scraping sometimes returns incorrect answers)
        time.sleep(1)
        # calculation
        res = buysig(ser).to_frame().T
        data = data.append(res, ignore_index=True)

ABOS: No SVF signals generated
ABOS: No S&F signals generated
ABSI: No SVF signals generated
ABSI: No S&F signals generated
ADGI: No SVF signals generated
ADGI: No S&F signals generated
AGRI: No SVF signals generated
AGRI: No S&F signals generated
ALKT: No S&F signals generated
ALZN: No SVF signals generated
ALZN: No S&F signals generated
ATAI: No SVF signals generated
ATAI: No S&F signals generated
AVAH: No S&F signals generated
BASE: No SVF signals generated
BASE: No S&F signals generated
BHG: No SVF signals generated
BHG: No S&F signals generated
BLND: No SVF signals generated
BLND: No S&F signals generated
BON: No S&F signals generated
CADL: No SVF signals generated
CADL: No S&F signals generated
CFLT: No SVF signals generated
CLVR: No S&F signals generated
CNM: No SVF signals generated
CNM: No S&F signals generated
CNTA: No SVF signals generated
CNTA: No S&F signals generated
CNVY: No S&F signals generated
COOK: No SVF signals generated
COOK: No S&F signals generated
CPOP: No SVF 

***
## Print & Save results

In [5]:
# display results
display(data[(data["dist_svf"] < 1) | (data["dist_saf"] < 1)].drop_duplicates(subset="ticker").reset_index(drop=True))
# save data
data[(data["dist_svf"] <= 1) | (data["dist_saf"] <= 1)].drop_duplicates(subset="ticker").reset_index(drop=True).to_excel("Screener-"+str(dt.date.today())+".xlsx")

,ticker,type,buy_svf,dist_svf,buy_saf,dist_saf,SMA50_bounce,SMA100_bounce,SMA200_bounce
0,AA,filtered,2021-09-24 00:00:00,15,2021-10-15 00:00:00,0,0,0,0
1,ABEV,filtered,2021-10-13 00:00:00,2,2021-10-15 00:00:00,0,0,0,0
2,ACHC,filtered,2021-10-15 00:00:00,0,2021-10-01 00:00:00,10,0,0,0
3,ADS,filtered,2021-10-15 00:00:00,0,2021-09-23 00:00:00,16,0,0,0
4,AFI,filtered,2021-10-15 00:00:00,0,2021-10-04 00:00:00,9,0,0,0
...,...,...,...,...,...,...,...,...,...
151,XENE,filtered,2021-10-15 00:00:00,0,2021-10-04 00:00:00,9,0,0,0
152,XP,filtered,2021-10-15 00:00:00,0,2021-08-24 00:00:00,37,0,0,0
153,XTNT,filtered,2021-10-15 00:00:00,0,2021-10-15 00:00:00,0,0,0,0
154,ZGNX,filtered,2021-10-15 00:00:00,0,2021-10-01 00:00:00,10,0,0,0
